# Data Wrangling with Twitter
Jessica Ertel

In [180]:
# data gathering 
import config
import os
import requests
import time
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

# standard data manipulation libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from datetime import datetime
from dateutil import parser

## Data Gathering
Using the tweet IDs from the WeRateDogs Twitter archive, I query the Twitter API for each tweet's JSON data and write the data to a text file.

In [181]:
# import WeRateDogs twitter archive (provided by Udacity)
archive = pd.read_csv('twitter-archive-enhanced.csv')

In [5]:
# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
consumer_key = config.CONSUMER_KEY
consumer_secret = config.CONSUMER_SECRET
access_token = config.ACCESS_TOKEN
access_secret = config.ACCESS_SECRET

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Gather IDs from WeRateDogs twitter archive
tweet_ids = archive.tweet_id.values


# Query Twitter's API for JSON data for each tweet ID
# Save each tweet's returned JSON as a new line in a .txt file
fails_dict = {}

with open('tweet_json.txt', 'w') as outfile:
    for tweet_id in tweet_ids:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print(f"Tweet ID: {tweet_id} no longer exists.") 
            fails_dict[tweet_id] = e
            pass

print(fails_dict)

Tweet ID: 888202515573088257 no longer exists.
Tweet ID: 873697596434513921 no longer exists.
Tweet ID: 872668790621863937 no longer exists.
Tweet ID: 872261713294495745 no longer exists.
Tweet ID: 869988702071779329 no longer exists.
Tweet ID: 866816280283807744 no longer exists.
Tweet ID: 861769973181624320 no longer exists.
Tweet ID: 856602993587888130 no longer exists.
Tweet ID: 851953902622658560 no longer exists.
Tweet ID: 845459076796616705 no longer exists.
Tweet ID: 844704788403113984 no longer exists.
Tweet ID: 842892208864923648 no longer exists.
Tweet ID: 837366284874571778 no longer exists.
Tweet ID: 837012587749474308 no longer exists.
Tweet ID: 829374341691346946 no longer exists.
Tweet ID: 827228250799742977 no longer exists.
Tweet ID: 812747805718642688 no longer exists.
Tweet ID: 802247111496568832 no longer exists.
Tweet ID: 779123168116150273 no longer exists.
Tweet ID: 775096608509886464 no longer exists.
Tweet ID: 771004394259247104 no longer exists.
Tweet ID: 770

In [16]:
# Load in json data line by line and turn into pandas df
data = []
with open('tweet-json.txt') as json_file:
    for row in json_file:
        json_data = json.loads(row)
        data.append({"tweet_id":json_data["id"],
                   "favorites":json_data["favorite_count"],
                   "retweets":json_data["retweet_count"],
                   "timestamp":json_data["created_at"]})

tweets = pd.DataFrame(data, columns=['tweet_id',
                                     'favorites',
                                     'retweets',
                                     'timestamp'])

In [17]:
tweets.head()

,tweet_id,favorites,retweets,timestamp
0,892420643555336193,39467,8853,Tue Aug 01 16:23:56 +0000 2017
1,892177421306343426,33819,6514,Tue Aug 01 00:17:27 +0000 2017
2,891815181378084864,25461,4328,Mon Jul 31 00:18:03 +0000 2017
3,891689557279858688,42908,8964,Sun Jul 30 15:58:51 +0000 2017
4,891327558926688256,41048,9774,Sat Jul 29 16:00:24 +0000 2017


In [18]:
# Save to csv
tweets.to_csv('twitter_scrape.csv', index=False)

In [182]:
# download image predictions programmatically (provided by Udacity)
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
preds = requests.get(url,auth=('user', 'pass'))

with open('image_predictions.tsv', 'wb') as file:
    file.write(preds.content)

image_preds = pd.read_csv('image_predictions.tsv', sep='\t')
image_preds.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


## Data Assessing
After gathering the data, I assess them visually and programmatically for quality and tidiness issues. I detect and document eight quality issues and two tidiness issues.

### Assessment
Tidiness  
Each variable forms a column and each observation forms a row. Ex: splitting info in one column into two. How many tables do you need?  
1. Combine the tables (archive, tweets, image_preds) and remove duplicated columns.
2. Address missing data.

Quality
1. Check data types
2. Check timestamp is the same
3. Each row is identified as doggo, floofer, pupper or puppo
4. If all three predictions are not breeds of dogs, then drop the row (likely not a dog).
5. Drop all ratings that do not have image predictions
6. Update names identified as 'just'
7. Update rating denominators (some are incorrectly pulled from notes)

## Data Cleaning: Tidiness

In [183]:
# import tweets
tweets = pd.read_csv('twitter_scrape.csv')
tweets.shape

(2354, 4)

In [184]:
image_preds.shape

(2075, 12)

In [185]:
archive.shape

(2356, 17)

### Merge Twitter Posts with Archive

In [186]:
# merge image preds, tweets and archive datasets
full = pd.merge(tweets, archive, on='tweet_id')
full.to_csv('full_raw.csv', index=False)

In [187]:
full = pd.read_csv('full_raw.csv')
full.head()

,tweet_id,favorites,retweets,timestamp_x,in_reply_to_status_id,in_reply_to_user_id,timestamp_y,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,39467,8853,Tue Aug 01 16:23:56 +0000 2017,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,33819,6514,Tue Aug 01 00:17:27 +0000 2017,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,25461,4328,Mon Jul 31 00:18:03 +0000 2017,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,42908,8964,Sun Jul 30 15:58:51 +0000 2017,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,41048,9774,Sat Jul 29 16:00:24 +0000 2017,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [188]:
full.shape

(2354, 20)

### Check for Nulls

In [189]:
# Check percentage of null values per column
full.isnull().sum() / len(full) * 100

tweet_id                       0.000000
favorites                      0.000000
retweets                       0.000000
timestamp_x                    0.000000
in_reply_to_status_id         96.686491
in_reply_to_user_id           96.686491
timestamp_y                    0.000000
source                         0.000000
text                           0.000000
retweeted_status_id           92.395922
retweeted_status_user_id      92.395922
retweeted_status_timestamp    92.395922
expanded_urls                  2.506372
rating_numerator               0.000000
rating_denominator             0.000000
name                           0.000000
doggo                          0.000000
floofer                        0.000000
pupper                         0.000000
puppo                          0.000000
dtype: float64

In [190]:
# Drop high % null columns and unnecessary columns
full.drop(columns=['in_reply_to_status_id', 
                   'in_reply_to_user_id', 
                   'retweeted_status_id', 
                   'retweeted_status_user_id',
                   'retweeted_status_timestamp',
                   'expanded_urls', 
                   'source'], inplace=True)

## Data Cleaning: Quality

### Filter to matching records

In [191]:
# There are more tweets than image predictions
image_preds['tweet_id'].nunique(), full['tweet_id'].nunique()

(2075, 2354)

In [192]:
# Filter the full dataset to include same IDs as image_preds
full = full[full['tweet_id'].isin(image_preds['tweet_id'])]
full.shape

(2073, 13)

### Change tweet_id to a string

In [193]:
full['tweet_id'] = full['tweet_id'].astype('str')

In [194]:
# Confirm change
full['tweet_id'].dtype

dtype('O')

### Change timestamp columns to datetime

In [195]:
# remove day of the week
full['timestamp_x'] = full['timestamp_x'].str.slice(4, )

In [196]:
# Change timestamp_x and y to datetime format
# illustrate two options for approaching this
full['timestamp_x'] = [datetime.strptime(row, '%b %d %H:%M:%S %z %Y') for row in full['timestamp_x']]
full['timestamp_y'] = [parser.parse(row) for row in full['timestamp_y']]


In [197]:
# Check data type is correct
full.dtypes

tweet_id                               object
favorites                               int64
retweets                                int64
timestamp_x               datetime64[ns, UTC]
timestamp_y           datetime64[ns, tzutc()]
text                                   object
rating_numerator                        int64
rating_denominator                      int64
name                                   object
doggo                                  object
floofer                                object
pupper                                 object
puppo                                  object
dtype: object

In [198]:
# Confirm timestamps match
(full['timestamp_x'] == full['timestamp_y']).value_counts()

True    2073
dtype: int64

In [199]:
# Drop duplicate column
full.drop(columns=['timestamp_y'], inplace=True)

### Merge dog stages into one column

In [200]:
# Can a dog identify in multiple stages?
full[(full['doggo']=='doggo')&(full['floofer']=='floofer')]

,tweet_id,favorites,retweets,timestamp_x,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
199,854010172552949760,17169,3433,2017-04-17 16:34:26+00:00,"At first I thought this was a shy doggo, but i...",11,10,None,doggo,floofer,None,None


In [201]:
# Iterate through dog stages and combine into one column
full['stage'] = full[['doggo', 'floofer', 'pupper', 'puppo']] \
.apply(lambda x: ','.join(x).replace('None,', '').replace(',None', ''), axis=1)

In [202]:
# Confirm value counts
full['stage'].value_counts()

None             1753
pupper            210
doggo              67
puppo              23
doggo,pupper       11
floofer             7
doggo,floofer       1
doggo,puppo         1
Name: stage, dtype: int64

In [203]:
# Drop original dog stage columns
full.drop(columns=['doggo', 'floofer', 'pupper', 'puppo'], inplace=True)

In [204]:
# Should have 22 columns
full.shape

(2073, 9)

### Update innaccurate names

In [205]:
# Replace unlikely dog names with 'None'
full['name'].replace(['just', 'a', 'the', 'an', 'by', 'his', 'O', 'my', 'all'], 'None', inplace=True)

In [206]:
# Confirm replace worked correctly
full[(full['name']=='just') | (full['name']=='a') | (full['name']=='my')]

,tweet_id,favorites,retweets,timestamp_x,text,rating_numerator,rating_denominator,name,stage


### Combine rating into one column

In [178]:
# full['rating'] = str(full['rating_numerator']).join(str(full['rating_denominator'])
#     str(full['rating_numerator']) + "/" + str(full['rating_denominator'])

In [152]:
full['rating_numerator'].value_counts()

12      473
10      429
11      413
13      283
9       151
8        95
7        52
14       40
5        34
6        32
3        19
4        16
2         9
1         5
0         2
80        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
143       1
420       1
1776      1
15        1
27        1
45        1
75        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

### Make all predictions lowercase

In [214]:
image_preds['p1'] = [row.lower() for row in image_preds['p1']]
image_preds['p2'] = [row.lower() for row in image_preds['p2']]
image_preds['p3'] = [row.lower() for row in image_preds['p3']]

In [217]:
# Confirm change
image_preds[['p1', 'p2', 'p3']].head()

,p1,p2,p3
0,welsh_springer_spaniel,collie,shetland_sheepdog
1,redbone,miniature_pinscher,rhodesian_ridgeback
2,german_shepherd,malinois,bloodhound
3,rhodesian_ridgeback,redbone,miniature_pinscher
4,miniature_pinscher,rottweiler,doberman


### Update column names

In [223]:
# Change the column names to make them more interpretable
image_preds.rename(columns= {'p1': 'pred_1',
                             'p2': 'pred_2',
                             'p3': 'pred_3',}, inplace=True)

## Analyze and Visualize
Analyze and visualize the wrangled data. I produce three (3) insights and one (1) visualization.

### Insights